In [ ]:
import math 
import pandas as pd
import numpy as np
import string 
from tqdm import tqdm
import sys
import matplotlib
import matplotlib.pyplot as plt
from google.colab import files
!pip install xlsxwriter
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')
#https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1

     |████████████████████████████████| 149 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 4.0 MB 32.2 MB/s 
     |████████████████████████████████| 1.2 MB 43.4 MB/s 
     |████████████████████████████████| 77 kB 6.3 MB/s 
     |████████████████████████████████| 895 kB 62.1 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
     |████████████████████████████████| 6.6 MB 33.6 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=c6a8522d562f92c6ee0ed526dd9c4831d5b9c2d8b6eae028af1b22d9b3e39034
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Upload relevant files

In [ ]:
print('\nUpload the Tripadvisor file')
File = files.upload()

print('\nUpload the Emotions file')
File = files.upload()


Upload the Tripadvisor file


Saving emotions_reviewID.csv to emotions_reviewID.csv

Upload the Emotions file


Saving TripAdvisor_US_data.csv to TripAdvisor_US_data.csv


In [ ]:
df = pd.read_csv("TripAdvisor_US_data.csv")
emotion = pd.read_csv("emotions_reviewID.csv")

In [ ]:
df = pd.merge(df,emotion,on='ReviewID')

In [ ]:
df = df.drop(columns='Unnamed: 0_y')
df['Similarity'] = range(0,len(df))
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124324 entries, 0 to 124323
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0_x        124324 non-null  int64  
 1   ReviewID            124324 non-null  float64
 2   ReviewDate          124324 non-null  object 
 3   ReviewStar          124324 non-null  float64
 4   DateOfStay          124324 non-null  object 
 5   ResponseDate        124324 non-null  object 
 6   TripType            124324 non-null  object 
 7   ValueScore          124324 non-null  float64
 8   ServiceScore        124324 non-null  float64
 9   Sleep QualityScore  124324 non-null  float64
 10  RoomsScore          124324 non-null  float64
 11  LocationScore       124324 non-null  float64
 12  CleanlinessScore    124324 non-null  float64
 13  City_Country        124324 non-null  object 
 14  ISO                 124324 non-null  object 
 15  YEAR                124324 non-nul

In [ ]:
for i in range(0,len(df)):
  if (df['ResponseText'][i]) == '.':
    df['ResponseText'][i] = 'Unresponsive'
  else:
    df['ResponseText'][i]  = df['ResponseText'][i] 

It may take a long time to run the entire code, the code block helps to get the solution in batches

In [ ]:
#n = round((len(df)/24))
#df = df[0:n]
#df = df.reset_index(drop=True)

In [ ]:
len(df)

20724

In [ ]:
df.iloc[:,16:].head()

,ReviewText,ResponseText,emotion,Similarity
0,we went to support my maternal aunt who lost h...,Our heartfelt condolences to your family aliso...,joy,103600
1,The cleaning staff started cleaning surroundin...,It is truly our goal to impress each guest wit...,joy,103601
2,The second day I was there I only needed the r...,I am so sorry to hear about the situation that...,joy,103602
3,Internet did not work. Ice machine had spider...,"christophermZ2687YQ,Thank you for taking the ...",joy,103603
4,The staff here is always helpful and quick wit...,"Dear Amy W,Thank you for being one of our fre...",joy,103604


In [ ]:
s1 = df['ReviewText'].tolist()
s2 = df['ResponseText'].tolist()
sentences = s1 + s2

In [ ]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
sentence_embed = []
for i in tqdm(range(0,len(sentences))):
  senemb = model.encode(sentences[i])
  sentence_embed.append(senemb)
sentence_embed = np.array(sentence_embed)
sentence_embed.shape

100%|██████████| 41448/41448 [3:30:09<00:00,  3.29it/s]


(41448, 768)

In [ ]:
sentence_embeddings = sentence_embed

In [ ]:
#sentence_embeddings = model.encode(sentences)

In [ ]:
sentence_embeddings.shape

(41448, 768)

In [ ]:
for i in tqdm(range(0,len(df))):
  df['Similarity'][i] = (cosine_similarity([sentence_embeddings[i]],[sentence_embeddings[len(df)+i]])) * 100

100%|██████████| 20724/20724 [00:16<00:00, 1250.71it/s]


In [ ]:
df.iloc[:,16:]

,ReviewText,ResponseText,emotion,Similarity
0,we went to support my maternal aunt who lost h...,Our heartfelt condolences to your family aliso...,joy,47
1,The cleaning staff started cleaning surroundin...,It is truly our goal to impress each guest wit...,joy,31
2,The second day I was there I only needed the r...,I am so sorry to hear about the situation that...,joy,44
3,Internet did not work. Ice machine had spider...,"christophermZ2687YQ,Thank you for taking the ...",joy,58
4,The staff here is always helpful and quick wit...,"Dear Amy W,Thank you for being one of our fre...",joy,60
...,...,...,...,...
20719,Excellent location for attending events at McD...,We are delighted you enjoyed your stay with us...,joy,67
20720,Came for our grandson's wedding. This location...,How wonderful! We are so happy we could provid...,joy,68
20721,I have been staying in hotels for work on a re...,We truly appreciate your patronage and feedbac...,joy,51
20722,Nice place to bring your pet. A lot of grass f...,Thank you so much for your feedback. We love o...,joy,68


In [ ]:
df.to_excel('Similarity.xlsx',engine='xlsxwriter') 
files.download('Similarity.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>